In [0]:
import pandas as pd
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
from google.colab import files
uploaded = files.upload()


!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c msk-redefining-cancer-treatment

In [0]:
!unzip training_variants.zip
!unzip test_variants.zip
!unzip training_text.zip
!unzip test_text.zip
!ls

In [0]:
train_df =pd.read_csv("training_variants")
train_df.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [0]:
train_text_df = pd.read_csv("training_text", sep="\|\|", engine="python",header= None, encoding='utf-8', skiprows=1, names=["ID", "Text"])
train_text_df.head()

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [0]:
training_merge_df = train_df.merge(train_text_df, left_on = "ID", right_on ="ID")
training_merge_df ['Text'] = training_merge_df["Text"]+" "+training_merge_df["Gene"]+" "+training_merge_df["Variation"]
training_merge_df.head(5)

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [0]:
training_merge_df.isnull().sum()

ID           0
Gene         0
Variation    0
Class        0
Text         5
dtype: int64

In [0]:
training_merge_df.dropna(axis=0, how='any',inplace=True)
training_merge_df.isnull().sum()

ID           0
Gene         0
Variation    0
Class        0
Text         0
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(training_merge_df, test_size = 0.2)
np.random.seed(0)
train

,ID,Gene,Variation,Class,Text
1529,1529,ALK,S1206R,7,"In the era of personalized medicine, understan..."
1623,1623,VHL,S65L,4,Abstract The von Hippel-Lindau (VHL) gene pr...
613,613,CDK6,A197T,4,Autosomal recessive primary microcephaly (MCPH...
495,495,TP53,T150I,1,Inheritance of germ-line mutant alleles of BRC...
1017,1017,TSC2,Q1554H,1,Tuberous sclerosis complex (TSC) is an autosom...
11,11,CBL,K382E,4,Noonan syndrome is an autosomal dominant conge...
325,325,ROS1,G1971E,2,The rapidly growing recognition of the role of...
2527,2527,BRCA1,H1402Y,5,BRCA1 is a tumour suppressor with pleiotropic ...
36,36,DICER1,G1809K,4,Abstract DICER1 plays a critical role in mic...
2239,2239,PTEN,L108P,4,Germline mutations in the phosphatase PTEN are...


In [0]:
X_train = train['Text'].values.astype('U')
X_test = test['Text'].values.astype('U')
y_train = train['Class'].values
y_test = test['Class'].values

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn import svm
# from sklearn.svm import SVC
from xgboost import XGBClassifier

In [0]:
#Set pipeline to build a complete text processing model with Vectorizer, Transformer and LinearSVC
text_clf = Pipeline([('vect', CountVectorizer()), 
                        ('tfidf', TfidfTransformer()),
                       ('clf', XGBClassifier())])
text_clf = text_clf.fit(X_train, y_train)

In [0]:
filename = 'model.pkl'
pickle.dump(text_clf, open(filename, 'wb'))

In [0]:
filename = 'model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [0]:
y_test_predicted = text_clf.predict(X_test)
np.mean(y_test_predicted == y_test)

0.6691729323308271

In [0]:
files.download("model.pkl")